In [ ]:
import livecellx.sample_data
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from napari.layers import Shapes


In [ ]:
dic_dataset_path = Path("../datasets/test_data_STAV-A549/DIC_data")
mask_dataset_path = Path("../datasets/test_data_STAV-A549/mask_data")
dic_dataset, mask_dataset = livecellx.sample_data.tutorial_three_image_sys(dic_dataset_path, mask_dataset_path, dir="../datasets")

In [ ]:
from livecellx.core.io_sc import prep_scs_from_mask_dataset
single_cells = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
from livecellx.model_zoo.segmentation.sc_correction import CorrectSegNet
# ckpt = r"./notebook_results/csn_models/model_v11_epoch=3282-test_loss=2.3688.ckpt"
ckpt = r"./notebook_results/csn_models/model_over_dropout_epoch=4092-test_loss=2.7959.ckpt"
model = CorrectSegNet.load_from_checkpoint(ckpt)
model = model.cuda()
model = model.eval()

In [ ]:
import napari
from livecellx.core.sc_seg_operator import ScSegOperator, create_sc_seg_napari_ui
viewer = napari.view_image(dic_dataset.to_dask(), name="dic_image", cache=True)

In [ ]:
sample_sc = single_cells[10]
sample_sc_seg_operator = ScSegOperator(sample_sc, viewer=viewer, csn_model=model)
sample_sc_seg_operator.create_sc_layer(contour_sample_num=30)


In [ ]:
sample_sc_seg_operator.save_seg_callback()

In [ ]:
sample_sc_seg_operator.csn_correct_seg_callback()

In [ ]:
padding_pixels = 40
create_ou_input_kwargs = {
            "padding_pixels": padding_pixels,
            "dtype": float,
            "remove_bg": False,
            "one_object": True,
            "scale": 0,
        }

output = sample_sc_seg_operator.correct_segment(model, create_ou_input_kwargs=create_ou_input_kwargs)
# plt.imshow(output[0].cpu().detach().numpy()[0])

In [ ]:
sample_sc_seg_operator.restore_sc_contour_callback()

In [ ]:
sample_sc_seg_operator.clear_sc_layer_callback()

In [ ]:
# # select a cell (by click or by set attr) and call the function below to
# sample_sc_seg_operator.csn_correct_seg_callback()

Remove sc operator shape layer

In [ ]:
sample_sc_seg_operator.remove_sc_layer()

Programmically replace sc mask with a new mask

In [ ]:
plt.imshow(mask[0])

In [ ]:
mask = (output[0].cpu().detach().numpy()[0] > 0.5).astype(np.uint8)
sample_sc_seg_operator.sc.update_sc_mask_by_crop(mask[0, ...], padding_pixels=padding_pixels)

In [ ]:
from livecellx.segment.utils import find_contours_opencv
from livecellx.core.datasets import SingleImageDataset


padding_pixels = 50
new_mask = (output[0].cpu().detach().numpy()[0] > 0.5).astype(np.uint8)[0]
new_sc = sample_sc_seg_operator.sc.copy()
new_sc.mask_dataset = SingleImageDataset(new_mask)
new_sc.contour = np.array(find_contours_opencv(new_mask)[0]) + new_sc.bbox[:2] - padding_pixels
new_sc_seg_operator = ScSegOperator(new_sc, viewer=viewer)
new_sc_seg_operator.create_sc_layer(contour_sample_num=20)

In [ ]:
create_sc_seg_napari_ui(sample_sc_seg_operator)

In [ ]:
sample_sc_seg_operator.csn_model = model
sample_sc_seg_operator.csn_correct_seg_callback()

In [ ]:
sample_sc_seg_operator.save_seg_callback()

In [ ]:
sample_sc_seg_operator.sc.get_label_mask_crop().shape

In [ ]:
sample_sc_seg_operator.sc.show_panel()

In [ ]:
len(sample_sc_seg_operator.shape_layer.data)